In [4]:
# Import modules
import numpy as np

# Import PySwarms
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [7]:
LIMITS = 100
DIMENSION = 30
SAMPLE_SIZE = 500
MAX_ITER = 10000

In [3]:
import logging

# Define custom operators
def compute_int_position(swarm, bounds, bh):
    """
    Custom position computation
    """
    try:
        temp_position = swarm.position.copy()
        temp_position += swarm.velocity

        if bounds is not None:
            temp_position = bh(temp_position, bounds)

        # This casting is the only change to the standard operator
        position = temp_position.astype(int)

    except AttributeError:
        print("Please pass a Swarm class")
        raise
    
    return position

def compute_int_velocity(swarm):
    try:
        swarm_size = swarm.position.shape
        c1 = swarm.options["c1"]
        c2 = swarm.options["c2"]
        w = swarm.options["w"]

        cognitive = (
                c1
                * np.random.uniform(0,1,swarm_size)
                * (swarm.pbest_pos - swarm.position)
        )
        social = (
                c2
                * np.random.uniform(0,1,swarm_size)
                * (swarm.best_pos - swarm.position)
        )

        # This casting is the only change to the standard operator
        updated_velocity = ((w * swarm.velocity) + cognitive + social).astype(int)

    except AttributeError:
        print("Please pass a Swarm class")
        raise

    return updated_velocity

# Define a custom topology. This is not 100% necessary, one could also use the
# built-in topologies. The following is the exact same as the Star topology
# but the compute_velocity and compute_position methods have been replaced
# by the custom ones
class IntStar(ps.backend.topology.Topology):
    def __init__(self, static=None, **kwargs):
        super(IntStar, self).__init__(static=True)

    def compute_gbest(self, swarm, **kwargs):
        try:
            if self.neighbor_idx is None:
                self.neighbor_idx = np.tile(
                        np.arange(swarm.n_particles), (swarm.n_particles, 1)
                        )
            if np.min(swarm.pbest_cost) < swarm.best_cost:
                best_pos = swarm.pbest_pos[np.argmin(swarm.pbest_cost)]
                best_cost = np.min(swarm.pbest_cost)
            else:
                best_pos, best_cost = swarm.best_pos, swarm.best_cost

        except AttributeError:
            print("Please pass a Swarm class")
            raise
        else:
            return best_pos, best_cost

    def compute_velocity(self, swarm):
        return compute_int_velocity(swarm)
    
    def compute_position(self, swarm, bounds, bh):
        return compute_int_position(swarm, bounds, bh)

# Define custom Optimizer class
class IntOptimizerPSO(ps.base.SwarmOptimizer):
    def __init__(self, n_particles, dimensions, options, bounds=None, initpos=None):
        super(IntOptimizerPSO, self).__init__(
                n_particles=n_particles,
                dimensions=dimensions,
                options=options,
                bounds=bounds,
                velocity_clamp=None,
                center=1.0,
                ftol=-np.inf,
                init_pos=initpos)
        self.reset()
        # The periodic strategy will leave the velocities on integer values
        self.bh = ps.backend.handlers.BoundaryHandler(strategy="periodic")
        self.top = IntStar()
        self.rep = ps.utils.Reporter(logger=logging.getLogger(__name__))
        self.name = __name__

    # More or less copy-paste of the optimize method of the GeneralOptimizerPSO
    def optimize(self, func, iters, n_processes=None):
        self.bh.memory = self.swarm.position

        self.swarm.pbest_cost = np.full(self.swarm_size[0], np.inf)
        pool = None if n_processes is None else np.Pool(n_processes)
        for i in self.rep.pbar(iters, self.name):
            self.swarm.current_cost = ps.backend.operators.compute_objective_function(self.swarm, func, pool=pool)
            self.swarm.pbest_pos, self.swarm.pbest_cost = ps.backend.operators.compute_pbest(self.swarm)
            best_cost_yet_found = self.swarm.best_cost
            self.swarm.best_pos, self.swarm.best_cost = self.top.compute_gbest(
                    self.swarm, **self.options
            )
            self.rep.hook(best_cost=self.swarm.best_cost)
            # Cou could also just use the custom operators on the next two lines
            self.swarm.velocity = self.top.compute_velocity(self.swarm) #compute_int_velocity(self.swarm)
            self.swarm.position = self.top.compute_position(self.swarm, self.bounds, self.bh) #compute_int_position(self.swarm, self.bounds, self.bh)
        final_best_cost = self.swarm.best_cost.copy()
        final_best_pos = self.swarm.pbest_pos[
                self.swarm.pbest_cost.argmin()
        ].copy()
        self.rep.log(
                "Optimization finished | best cost: {}, best pos: {}".format(
                    final_best_cost, final_best_pos
        ))
        if n_processes is not None:
            pool.close()

        return final_best_cost, final_best_pos

# Define objective function with the cost decorator allows the defintion of the
# objective function for one particle
@ps.cost
def parab(X):
    cost = X[0]**2 + X[1]**2
    return cost

@ps.cost
def esfera(sol):
    sum = 0
    for xi in sol:
        sum += xi ** 2
    return sum

# Create bounds
max_bound = LIMITS * np.ones(DIMENSION)
min_bound = - max_bound
bounds = (min_bound, max_bound)

#initpos = np.array([[-7,4],[12,23],[-4,8],[-9,-7],[-3,2]]) # Initial positions of the particles
opt = IntOptimizerPSO(n_particles=SAMPLE_SIZE,
                      dimensions=DIMENSION,
                      options={"c1": 1.3, "c2": 1.5, "w": 0.1},
                      bounds=bounds)

c, p = opt.optimize(esfera, iters=MAX_ITER)

__main__: 100%|██████████|10000/10000, best_cost=1752.0
2024-03-20 12:44:42,133 - __main__ - INFO - Optimization finished | best cost: 1752.0, best pos: [  1.   0.   3.   0.   3.   7.  11. -21.   7.  -2. -10. -12.  -5.   0.
   3.   2.   1.  11.  10.  13.  16.  -3.   6.   7.  -1.   5.  -1.  -1.
  -3.  -3.]


In [4]:
# Create bounds
max_bound = LIMITS * np.ones(DIMENSION)
min_bound = - max_bound
bounds = (min_bound, max_bound)

In [5]:
def esfera(sol):
    sum = 0
    for xi in sol:
        sum += xi ** 2
    return sum

In [1]:
import networkx as nx
G = nx.read_edgelist("brock200_1_complement.edges", nodetype=int)

In [2]:
def subsets(a):
  l = len(a)
  n = l - 1
  x = [0] * n
  f = [j for j in range(l)]
  r = list()
  
  while n:
    m = [0] + x # insert 0 at the beginning of the bit string forms
    subset = [a[i] for i in range(l) if m[i] == 1] # generate the subset
    
    # and use the subset to generate the subsets for when 1 is inserted at the beginning
    k = [a[0]] + subset
    
    r.append(subset) # add to the table all subsets of bits strigs with prefix 0
    r.append(k) # add to the table all subsets of bits strings with prefix 1
    
    # initialization
    # this is where we choose which j we want to change
    j = f[0]
    f[0] = 0
    
    # if j == n we terminate because we have all the possible bit strings
    if j == n:
      break
     
    # update our array when j != n
    f[j] = f[j + 1]
    f[j + 1] = j + 1
    
    x[j] = 1 - x[j] # complement coordinate j and return to line 8 if j != n
  
  return r

In [5]:
import random

def is_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            return False
    return True

def naive_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            node = random.choice([u, v])
            sol[node - 1] = 1
    return

@ps.cost
def fitness(sol):
    n = G.order()
    if is_cover(sol, G):
        return sum(sol)
    else:
        if sum(sol) >= (2 * n // 3):
            zeros_index = []
            for i in range(len(sol)):
                if sol[i] == 0:
                    zeros_index.append(i)
                    #sol[i] = 1
                    #if is_cover(sol, G):
                    #    print(sol)
                    #    break
                    #sol[i] = 0
            r = subsets(zeros_index)
            r.sort(key=len)
            # r[0] es el conjunto vacío
            for subset in r[1:]:
                for index in subset:
                    sol[index] = 1 # Esto modifica sol porque se pasa por referencia
                if is_cover(sol, G):
                    return sum(sol)
                for index in subset:
                    sol[index] = 0
            #print("calavera 1")
            return 0
        else:
            naive_cover(sol, G)
            #print("calavera 2")
            return sum(sol)
        

@ps.cost
def fitness2(sol):
    n = G.order()
    if is_cover(sol, G):
        return sum(sol)
    else:
        naive_cover(sol, G)
        return sum(sol)

In [8]:
# Set-up hyperparameters
options = {'c1': 0.7, 'c2': 0.7, 'w':0.2, 'k':10, 'p':1}

# Call instance of PSO
optimizer = ps.discrete.binary.BinaryPSO(n_particles=100, dimensions=G.order(), options=options)

# Perform optimization
cost, pos = optimizer.optimize(fitness2, iters=1000)

2024-03-20 19:55:33,654 - pyswarms.discrete.binary - INFO - Optimize for 1000 iters with {'c1': 0.7, 'c2': 0.7, 'w': 0.2, 'k': 10, 'p': 1}
pyswarms.discrete.binary: 100%|██████████|1000/1000, best_cost=188
2024-03-20 19:57:17,967 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 188.0, best pos: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1]
